# Q2: Completion Analysis

**Purpose:** Assess pipeline efficiency beyond volume counts

**Three questions:**
1. Which phases show the highest completion rates?
2. What termination patterns emerge across the pipeline?
3. How long do trials take to complete?

**Eligibility rules:**
- **Scope:** Interventional trials only (excludes "Not Applicable" from phase-specific analysis)
- **Minimum:** Phase groups with ≥10 trials
- **Status:** All statuses included (completed, active, terminated, withdrawn, suspended)

**What this analysis does NOT cover:**
- Enrollment adequacy (Q3)
- Geographic distribution (Q4)
- Duration trends over time (Q5)

In [ ]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

# Database connection
DB_PATH = Path('../data/database/clinical_trials.db')
conn = sqlite3.connect(str(DB_PATH))

In [ ]:
# Dataset snapshot
snapshot = pd.read_sql_query("""
    SELECT 
        COUNT(*) as n_studies,
        COUNT(CASE WHEN status = 'COMPLETED' THEN 1 END) as n_completed,
        COUNT(CASE WHEN status IN ('TERMINATED', 'WITHDRAWN', 'SUSPENDED') THEN 1 END) as n_stopped
    FROM studies
""", conn)

n_studies = int(snapshot['n_studies'].iloc[0])
n_completed = int(snapshot['n_completed'].iloc[0])
n_stopped = int(snapshot['n_stopped'].iloc[0])
overall_completion = round(n_completed / n_studies * 100, 1)

print(f"Dataset: {n_studies:,} trials · {n_completed:,} completed ({overall_completion}%) · {n_stopped:,} stopped")

---

## 1. Completion Rates by Phase

**Question:** Which phases successfully complete vs terminate or withdraw?

In [ ]:
# Load completion rates data
with open('../sql/queries/q2_completion_rates_by_phase.sql', 'r') as f:
    query_completion = f.read()

df_completion = pd.read_sql_query(query_completion, conn)
df_completion.head(10)

In [ ]:
# Filter to interventional phases only (exclude "Not Applicable")
df_interventional = df_completion[df_completion['phase_group'] != 'Not Applicable'].copy()

# Calculate stopped = terminated + withdrawn + suspended
df_interventional['stopped'] = df_interventional['terminated'] + df_interventional['withdrawn'] + df_interventional['suspended']
df_interventional['stopped_rate'] = df_interventional['stopped'] / df_interventional['total_trials'] * 100
df_interventional['active_rate'] = df_interventional['active'] / df_interventional['total_trials'] * 100

# Stacked bar chart: completion vs termination vs active
fig_completion = go.Figure()

fig_completion.add_trace(go.Bar(
    name='Completed',
    x=df_interventional['phase_group'],
    y=df_interventional['completion_rate'],
    marker_color='#2563eb',
    text=[f"{v:.1f}%" for v in df_interventional['completion_rate']],
    textposition='inside',
    textfont=dict(color='white', size=11),
    hovertemplate='<b>%{x}</b><br>Completed: %{y:.1f}%<extra></extra>'
))

fig_completion.add_trace(go.Bar(
    name='Stopped',
    x=df_interventional['phase_group'],
    y=df_interventional['stopped_rate'],
    marker_color='#dc2626',
    text=[f"{v:.1f}%" if v >= 5 else '' for v in df_interventional['stopped_rate']],
    textposition='inside',
    textfont=dict(color='white', size=11),
    hovertemplate='<b>%{x}</b><br>Stopped: %{y:.1f}%<extra></extra>'
))

fig_completion.add_trace(go.Bar(
    name='Active',
    x=df_interventional['phase_group'],
    y=df_interventional['active_rate'],
    marker_color='#94a3b8',
    text=[f"{v:.1f}%" if v >= 5 else '' for v in df_interventional['active_rate']],
    textposition='inside',
    textfont=dict(color='white', size=11),
    hovertemplate='<b>%{x}</b><br>Active: %{y:.1f}%<extra></extra>'
))

fig_completion.update_layout(
    title='<b>Completion Rates by Development Phase</b>',
    barmode='stack',
    xaxis=dict(title=None, tickfont=dict(size=11)),
    yaxis=dict(title='Percentage of trials', range=[0, 100]),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    height=500,
    template='plotly_white',
    font=dict(family="Arial", color="#374151"),
    margin=dict(t=60, b=50)
)
fig_completion.show()

In [ ]:
# Key statistics
highest_completion = df_interventional.loc[df_interventional['completion_rate'].idxmax()]
lowest_completion = df_interventional.loc[df_interventional['completion_rate'].idxmin()]

key_stats = pd.DataFrame([
    {'Metric': 'Highest completion rate', 'Value': f"{highest_completion['phase_group']}: {highest_completion['completion_rate']:.1f}%"},
    {'Metric': 'Lowest completion rate', 'Value': f"{lowest_completion['phase_group']}: {lowest_completion['completion_rate']:.1f}%"},
    {'Metric': 'Total interventional trials', 'Value': f"{df_interventional['total_trials'].sum():,}"},
    {'Metric': 'Overall interventional completion', 'Value': f"{df_interventional['completed'].sum() / df_interventional['total_trials'].sum() * 100:.1f}%"},
])

key_stats

In [ ]:
# Full completion statistics
completion_stats = df_interventional[['phase_group', 'total_trials', 'completion_rate', 'termination_rate', 'withdrawal_rate']].copy()
completion_stats.columns = ['Phase', 'Total Trials', 'Completion %', 'Termination %', 'Withdrawal %']
completion_stats

### What we see

- **Phase 1 shows highest completion rate** at 68.7%, reflecting simpler safety endpoints
- **Phase 1/2 and Phase 2 show lower completion** (45.0% and 55.1%), suggesting efficacy testing introduces attrition
- **Phase 3 recovers to 64.2%**, indicating trials reaching late stage have higher probability of completion

### Implication

Mid-stage attrition suggests operational challenges in Phase 2 trials. **Q3 should examine enrollment performance** to determine whether recruitment difficulties contribute to early terminations.

---

## 2. Termination Patterns

**Question:** Which phases show highest termination rates and at what enrollment scale?

In [ ]:
# Load termination patterns
with open('../sql/queries/q2_termination_patterns.sql', 'r') as f:
    query_termination = f.read()

df_termination = pd.read_sql_query(query_termination, conn)
df_termination.head(10)

In [ ]:
# Focus on terminated/withdrawn trials only, exclude "Not Applicable"
df_stopped = df_termination[
    (df_termination['failure_type'].isin(['TERMINATED', 'WITHDRAWN'])) &
    (df_termination['phase_group'] != 'Not Applicable')
].copy()

# Pivot for grouped bar chart
pivot_stopped = df_stopped.pivot_table(
    index='phase_group',
    columns='failure_type',
    values='trial_count',
    fill_value=0
)

fig_termination = go.Figure()

if 'TERMINATED' in pivot_stopped.columns:
    fig_termination.add_trace(go.Bar(
        name='Terminated',
        x=pivot_stopped.index,
        y=pivot_stopped['TERMINATED'],
        marker_color='#dc2626',
        text=[f"{int(v)}" for v in pivot_stopped['TERMINATED']],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Terminated: %{y}<extra></extra>'
    ))

if 'WITHDRAWN' in pivot_stopped.columns:
    fig_termination.add_trace(go.Bar(
        name='Withdrawn',
        x=pivot_stopped.index,
        y=pivot_stopped['WITHDRAWN'],
        marker_color='#f97316',
        text=[f"{int(v)}" for v in pivot_stopped['WITHDRAWN']],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Withdrawn: %{y}<extra></extra>'
    ))

fig_termination.update_layout(
    title='<b>Termination and Withdrawal Counts by Phase</b>',
    barmode='group',
    xaxis=dict(title=None, tickfont=dict(size=11)),
    yaxis=dict(title='Number of trials'),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    height=500,
    template='plotly_white',
    font=dict(family="Arial", color="#374151"),
    margin=dict(t=60, b=50, r=50)
)
fig_termination.show()

In [ ]:
# Key statistics
total_terminated = pivot_stopped['TERMINATED'].sum() if 'TERMINATED' in pivot_stopped.columns else 0
total_withdrawn = pivot_stopped['WITHDRAWN'].sum() if 'WITHDRAWN' in pivot_stopped.columns else 0
phase_with_most_term = pivot_stopped['TERMINATED'].idxmax() if 'TERMINATED' in pivot_stopped.columns else 'N/A'
most_term_count = int(pivot_stopped['TERMINATED'].max()) if 'TERMINATED' in pivot_stopped.columns else 0

key_stats_term = pd.DataFrame([
    {'Metric': 'Total terminated (interventional)', 'Value': f"{int(total_terminated):,}"},
    {'Metric': 'Total withdrawn (interventional)', 'Value': f"{int(total_withdrawn):,}"},
    {'Metric': 'Phase with most terminations', 'Value': f"{phase_with_most_term}: {most_term_count}"},
])

key_stats_term

In [ ]:
# Enrollment at termination
termination_enrollment = df_stopped[['phase_group', 'failure_type', 'trial_count', 'avg_enrollment_at_stop']].copy()
termination_enrollment.columns = ['Phase', 'Failure Type', 'Count', 'Avg Enrollment at Stop']
termination_enrollment

### What we see

- **Phase 2 leads in absolute terminations** at 130 trials, consistent with lower completion rates
- **Phase 3 terminations occur at higher enrollment** (avg ~1170), suggesting later-stage failures waste more resources
- **Withdrawn trials show lower enrollment**, indicating earlier decision points

### Implication

Phase 2 and Phase 3 terminations represent significant resource loss. **Q4 should examine geographic distribution** to identify whether site selection patterns correlate with termination risk.

---

## 3. Time-to-Completion

**Question:** How long do trials take to complete across phases?

In [ ]:
# Load duration data
with open('../sql/queries/q2_time_to_completion.sql', 'r') as f:
    query_duration = f.read()

df_duration = pd.read_sql_query(query_duration, conn)
df_duration.head(10)

In [ ]:
# Filter to interventional phases only
df_duration_interventional = df_duration[df_duration['phase_group'] != 'Not Applicable'].copy()

# Bar chart with range bars
fig_duration = go.Figure()

# Add bars for average duration
fig_duration.add_trace(go.Bar(
    x=df_duration_interventional['phase_group'],
    y=df_duration_interventional['avg_years_to_complete'],
    marker_color='#2563eb',
    text=[f"{v:.1f}y" for v in df_duration_interventional['avg_years_to_complete']],
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>Average: %{y:.1f} years<extra></extra>'
))

# Add error bars for min-max range
fig_duration.add_trace(go.Scatter(
    x=df_duration_interventional['phase_group'],
    y=df_duration_interventional['avg_years_to_complete'],
    error_y=dict(
        type='data',
        symmetric=False,
        array=df_duration_interventional['max_years'] - df_duration_interventional['avg_years_to_complete'],
        arrayminus=df_duration_interventional['avg_years_to_complete'] - df_duration_interventional['min_years'],
        color='#94a3b8',
        thickness=1.5,
        width=4
    ),
    mode='markers',
    marker=dict(size=0.1, color='rgba(0,0,0,0)'),
    showlegend=False,
    hoverinfo='skip'
))

fig_duration.update_layout(
    title='<b>Average Time-to-Completion by Phase</b>',
    xaxis=dict(title=None, tickfont=dict(size=11)),
    yaxis=dict(title='Years to completion', rangemode='tozero'),
    height=500,
    template='plotly_white',
    font=dict(family="Arial", color="#374151"),
    margin=dict(t=60, b=50, r=50)
)
fig_duration.show()

In [ ]:
# Key statistics
longest_phase = df_duration_interventional.loc[df_duration_interventional['avg_years_to_complete'].idxmax()]
shortest_phase = df_duration_interventional.loc[df_duration_interventional['avg_years_to_complete'].idxmin()]

key_stats_duration = pd.DataFrame([
    {'Metric': 'Longest average duration', 'Value': f"{longest_phase['phase_group']}: {longest_phase['avg_years_to_complete']:.1f} years"},
    {'Metric': 'Shortest average duration', 'Value': f"{shortest_phase['phase_group']}: {shortest_phase['avg_years_to_complete']:.1f} years"},
    {'Metric': 'Total completed trials analyzed', 'Value': f"{df_duration_interventional['completed_trials'].sum():,}"},
])

key_stats_duration

In [ ]:
# Duration statistics
duration_stats = df_duration_interventional[['phase_group', 'completed_trials', 'avg_years_to_complete', 'min_years', 'max_years']].copy()
duration_stats.columns = ['Phase', 'Completed Trials', 'Avg Years', 'Min Years', 'Max Years']
duration_stats

### What we see

- **Phase 1/2 and Phase 2 take longest** at 4.0 and 3.4 years respectively
- **Phase 1 completes fastest** at 1.9 years, consistent with simpler safety endpoints
- **Wide ranges across all phases** suggest high variability in trial duration

### Implication

Combined with lower completion rates, Phase 2 duration represents a double bottleneck. **Q5 should analyze whether duration is increasing over time** to assess pipeline velocity trends.

---

## Summary

**What this analysis establishes:**

1. **Pipeline efficiency varies by phase:** Phase 1 shows highest completion (68.7%); Phase 1/2 and Phase 2 show attrition
2. **Termination concentrates in Phase 2:** Highest absolute terminations among interventional phases
3. **Duration bottleneck in mid-stage:** Phase 1/2 and Phase 2 take 3-4 years on average

**Why subsequent analyses are needed:**

- **Q3 (Enrollment):** Lower completion rates don't reveal whether trials struggle to recruit—need enrollment adequacy metrics
- **Q4 (Geography):** Termination patterns ignore location—need to understand site selection impact
- **Q5 (Duration trends):** Current duration metrics are static—need to assess whether trials are getting longer over time

---

## Data Limitations

**Status classification:**
- Termination reasons not available in dataset (cannot distinguish safety vs futility vs operational)
- Status reflects point-in-time snapshot (active trials may later complete or terminate)

**Duration measurement:**
- Completion date may reflect database update lag, not actual study end
- Missing completion dates exclude many trials from duration analysis
- Only includes trials with valid start_date < completion_date

**Enrollment context:**
- Enrollment at termination doesn't indicate whether target was met or missed
- Enrollment data availability varies across trials

In [ ]:
# Close connection
conn.close()